In [1]:
import pandas as pd

## Upload the data as a data frame

In [2]:
synergy_logistics_info_df = pd.read_csv('synergy_logistics_database.csv', usecols = lambda x: x != 'register_id')
synergy_logistics_info_df

,direction,origin,destination,year,date,product,transport_mode,company_name,total_value
0,Exports,Japan,China,2015,31/01/15,Cars,Sea,Honda,33000000
1,Exports,Japan,China,2015,01/02/15,Cars,Sea,Honda,16000000
2,Exports,Japan,China,2015,02/02/15,Cars,Sea,Honda,29000000
3,Exports,Japan,China,2015,03/02/15,Cars,Sea,Honda,14000000
4,Exports,Japan,China,2015,04/02/15,Cars,Sea,Honda,17000000
...,...,...,...,...,...,...,...,...,...
19051,Imports,Japan,Singapore,2020,27/06/20,Gas turbines,Sea,Union Energy Co,1000000
19052,Imports,Malaysia,Singapore,2020,28/06/20,Gas turbines,Sea,Union Energy Co,2000000
19053,Imports,Malaysia,Singapore,2020,29/06/20,Gas turbines,Sea,Union Energy Co,33000000
19054,Imports,Malaysia,Singapore,2020,30/06/20,Gas turbines,Sea,Union Energy Co,13000000


# Option 1 .- Import and Export routes
## Obtain the 10 most used routes

In [3]:
option_1_df =  synergy_logistics_info_df.copy()
option_1_df.drop(columns = ['product', 'transport_mode', 'date', 'company_name'], inplace=True)

# divide in two df, one for imports and the other for exports
option_1_df_exports = option_1_df[option_1_df['direction'] == 'Exports']
option_1_df_imports = option_1_df[option_1_df['direction'] == 'Imports']

def get_routes(dataframe):
    '''

    '''
    # Get the yearly sum of the sales per route
    dataframe = dataframe.drop(columns='direction')
    dataframe = dataframe.groupby(['origin', 'destination', 'year'], sort=False).agg(
        total_value=pd.NamedAgg(column='total_value', aggfunc='sum'),
        total_sales=pd.NamedAgg(column='total_value', aggfunc='count'))

    # Get the mean total_value per route
    dataframe = dataframe.reset_index().drop(columns='year')
    dataframe = dataframe.groupby(['origin', 'destination'], sort=False).agg(
        avg_total_value=pd.NamedAgg(column='total_value', aggfunc='mean'),
        total_sales=pd.NamedAgg(column='total_sales', aggfunc='sum'))
    dataframe = dataframe.reset_index()

    return dataframe

def sort_routes_by_index(dataframe):
    """

    """
    # We need to normalize using feature scaling the avg_total_value and total_sales,
    # add them together by using a weighted average and sort them in descending order

    # Get the min and max values of each column
    max_avg_total_value = dataframe['avg_total_value'].max()
    max_total_sales = dataframe['total_sales'].max()
    min_avg_total_value = dataframe['avg_total_value'].min()
    min_total_sales = dataframe['total_sales'].min()

    # Use this statistics to normalize and create an index to sort the routes
    scaled_avg_total_value = (dataframe['avg_total_value'] - min_avg_total_value) / (
                max_avg_total_value - min_avg_total_value)
    scaled_total_sales = (dataframe['total_sales'] - min_total_sales) / (max_total_sales - min_total_sales)

    # We perform an arithmetic mean to get the index
    dataframe['index'] = (scaled_avg_total_value + scaled_total_sales) / 2

    dataframe = dataframe.sort_values(by='index', ascending=False)

    return dataframe

In [4]:
option_1_df_exports = get_routes(option_1_df_exports)
option_1_df_imports = get_routes(option_1_df_imports)

option_1_df_exports = sort_routes_by_index(option_1_df_exports)
option_1_df_imports = sort_routes_by_index(option_1_df_imports)

option_1_df_exports = sort_routes_by_index(option_1_df_exports).reset_index(drop=True)
option_1_df_imports = sort_routes_by_index(option_1_df_imports).reset_index(drop=True)

option_1_df_exports.index = option_1_df_exports.index + 1
option_1_df_imports.index = option_1_df_imports.index + 1

print(option_1_df_exports[:10])
print('\n')
print(option_1_df_imports[:10])

         origin  destination  avg_total_value  total_sales     index
1         China       Mexico     4.083333e+09          330  0.829592
2   South Korea      Vietnam     1.719252e+09          497  0.710520
3   Netherlands      Belgium     8.095355e+08          437  0.537902
4           USA  Netherlands     2.580468e+08          436  0.469352
5        Canada       Mexico     1.690000e+09          261  0.466122
6   South Korea        Japan     1.531333e+09          279  0.465061
7        France      Belgium     1.846023e+09          223  0.446451
8           USA       Canada     2.412000e+09          136  0.426979
9         Japan       Brazil     8.420388e+08          306  0.408208
10      Germany        Italy     2.270500e+09          130  0.403530


       origin           destination  avg_total_value  total_sales     index
1   Singapore              Thailand     1.004250e+09          273  0.944990
2       China              Thailand     9.577500e+08          200  0.787011
3       Jap

# Option 2 .- Transportation means
## Obtain the income per route and tell which one generates the least

In [63]:
option_2_df =  synergy_logistics_info_df.copy()
option_2_df = option_2_df[['transport_mode', 'direction', 'total_value']]
option_2_df.groupby(['transport_mode', 'direction']).sum().sort_values(by='total_value', ascending=False)

total_value
transport_mode direction             
Sea            Exports    65592622000
               Imports    34938000000
Rail           Exports    34505043000
Air            Exports    32785147000
Road           Exports    27280486000
Rail           Imports     9123000000
Road           Imports     5990000000
Air            Imports     5477000000

In [60]:
option_2_df.groupby(['transport_mode']).sum().sort_values(by='total_value', ascending=False)

,total_value
transport_mode,
Sea,100530622000
Rail,43628043000
Air,38262147000
Road,33270486000


# Option 3 .- Total value imports and exports
## Get the countries that give 80% of revenue 

In [45]:
option_3_df =  synergy_logistics_info_df.copy()

# We will get the origin or destination and create a new column based on that
option_3_df['country'] = option_3_df.apply(lambda row : row['origin'] if row['direction'] == 'Exports' else row['destination'], axis=1)
option_3_df = option_3_df[['country', 'total_value']]
option_3_df = option_3_df.groupby('country').sum()
option_3_df = option_3_df.sort_values(by='total_value', ascending=False)
option_3_df['cumulative_percentage'] = 100 * option_3_df['total_value'].cumsum()/option_3_df['total_value'].sum()
option_3_df.where(option_3_df['cumulative_percentage']<82).dropna()

In [50]:
option_3_df = option_3_df.groupby('country').sum()
option_3_df = option_3_df.sort_values(by='total_value', ascending=False)

In [52]:
option_3_df['cumulative_percentage'] = 100 * option_3_df['total_value'].cumsum()/option_3_df['total_value'].sum()

In [58]:
option_3_df.where(option_3_df['cumulative_percentage']<82).dropna()

,total_value,cumulative_percentage
country,,
China,3.554905e+10,16.481447
USA,2.211331e+10,26.733741
France,1.861433e+10,35.363821
Japan,1.777698e+10,43.605681
Germany,1.645223e+10,51.233357
South Korea,1.462115e+10,58.012094
Thailand,1.374500e+10,64.384628
Russia,1.322300e+10,70.515148
Canada,1.173600e+10,75.956258
